Test if the value inside the 1-D stiffness matrix would change after interpolation and restriction.

Conclusion: the multigrid coarse grid matrix is not equal to the original matrix, and multigrid matrix is not symmetric.

In [11]:
import numpy as np
import random

In [12]:
def shape(p):
    N = 0.5*np.array([1-p,1+p])
    dNdp = 0.5*np.array([-1,1])
    return N, dNdp

In [13]:
# parameters
nshape = 2
nele = 8
L = 0.01
nn = nele + 1
mesh = np.linspace(0, L, nn)
node_list = np.arange(nn)
conn = np.stack((node_list[:-1],node_list[1:]), axis=-1)
qpts = np.array([[-1, 1], [1, 1]])/np.sqrt(3)

In [14]:
K = np.zeros((nn,nn))
F = np.zeros((nn,1))
for c in conn:
    xe = mesh[c]
    Ke = np.zeros((2,2))
    Fe = np.zeros((2,))
    for q in qpts:
        N, dNdp = shape(q[0])
        x = np.dot(xe, N)
        J = np.dot(xe, dNdp) 
        B = dNdp/J
        A = 3.14*(0.5+(0.1-0.5)*x/L)**2
        sA = 0.1*100**2/A
        Ke += A*J*np.dot(B.T,B)*q[1]
        Fe += J*sA*N*q[1]
    K[np.ix_(c,c)] += Ke
    F[c] += Fe.reshape((2,1))

In [15]:
# prescribed temperature boundary
bc = [0,nn-1]
K[bc,:] = 0
K[np.ix_(bc,bc)] = np.eye(2)
F[bc] = np.array([0,8]).reshape((2,1))

In [16]:
#np.savetxt('K_5.txt', K, fmt='%1.3e')
#np.savetxt('F_5.txt', F, fmt='%1.3e')

In [17]:
def interp_matrix(n):
    stencil = [1, 2, 1]
    m = len(stencil)
    a = np.zeros((n * (m - 1) + 1, n))
    for i in range(n):
        a[i * (m - 1):i * (m - 1) + m, i] = stencil
    return 0.5*a[1:-1]

In [18]:
I = interp_matrix(5)
I

array([[1. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 1. ]])

In [19]:
I.T

array([[1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. ]])

In [20]:
print(K.shape)
print(I.shape)
print(I.T.shape)

(9, 9)
(9, 5)
(5, 9)


In [22]:
np.dot(I.T,np.dot(K,I))/2

array([[ 311.7110401 ,  278.57920289,    0.        ,    0.        ,
           0.        ],
       [ 442.34268224, 1169.91179347,  268.3062171 ,    0.        ,
           0.        ],
       [   0.        ,  268.3062171 ,  662.3054368 ,  137.77886824,
           0.        ],
       [   0.        ,    0.        ,  137.77886824,  299.72946775,
          50.76063567],
       [   0.        ,    0.        ,    0.        ,   39.27906331,
          28.90178424]])